### Download Eumetsat data

The following notebook shows a method to download data from Eumetsat using its API.

You will need an account with Eumetsat, and this will give you a consumer key, and consumer secret.

Quick start with Python page
https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/pages/1760198661/Quickstart+for+Python

Example Python code
https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/pages/1760100382/Examples

#### Set Up

Import EUMDAC to begin.

In [1]:
import eumdac
import datetime
import shutil

Authorize with your personal API keys:
Insert your personal key and secret into the single quotes

In [2]:

consumer_key = ''
consumer_secret = ''

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

print(f"This token '{token}' expires {token.expiration}")

This token 'eb951b95-3968-3c90-ad7f-4afc890e8f68' expires 2022-05-31 11:58:02.967056


#### List all available data

In [3]:
# Get all collection you can explore
datastore = eumdac.DataStore(token)
datastore.collections

[<class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:MSG15-RSS),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:RSS-CLM),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:HRSEVIRI),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:CLM),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0411),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0080),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0407),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0409),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:HRSEVIRI-IODC),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:CLM-IODC),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0081),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0082),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:AVHRRL1),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:AMSUL1),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:MHSL1),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:S

In [4]:
#  Now we can just loop over the objects and print the ID and title of each item.
for collection in datastore.collections:
    print(f"{collection} - {collection.title}")

EO:EUM:DAT:MSG:MSG15-RSS - Rapid Scan High Rate SEVIRI Level 1.5 Image Data - MSG
EO:EUM:DAT:MSG:RSS-CLM - Rapid Scan Cloud Mask - MSG
EO:EUM:DAT:MSG:HRSEVIRI - High Rate SEVIRI Level 1.5 Image Data - MSG - 0 degree
EO:EUM:DAT:MSG:CLM - Cloud Mask - MSG - 0 degree
EO:EUM:DAT:0411 - SLSTR Level 1B Radiances and Brightness Temperatures - Sentinel-3
EO:EUM:DAT:0080 - MVIRI Level 1.5 Climate Data Record Release 1 - MFG - 0 degree
EO:EUM:DAT:0407 - OLCI Level 2 Ocean Colour Full Resolution - Sentinel-3
EO:EUM:DAT:0409 - OLCI Level 1B Full Resolution - Sentinel-3
EO:EUM:DAT:MSG:HRSEVIRI-IODC - High Rate SEVIRI Level 1.5 Image Data - MSG - Indian Ocean
EO:EUM:DAT:MSG:CLM-IODC - Cloud Mask - MSG - Indian Ocean
EO:EUM:DAT:0081 - MVIRI Level 1.5 Climate Data Record Release 1 - MFG - 57 degree
EO:EUM:DAT:0082 - MVIRI Level 1.5 Climate Data Record Release 1 - MFG - 63 degree
EO:EUM:DAT:METOP:AVHRRL1 - AVHRR Level 1B - Metop - Global
EO:EUM:DAT:METOP:AMSUL1 - AMSU-A Level 1B - Metop - Global
EO:EUM

#### Select the data collection to download

In [5]:
# Select a collection
# EO:EUM:DAT:0407 - OLCI Level 2 Ocean Colour Full Resolution - Sentinel-3
selected_collection = datastore.get_collection('EO:EUM:DAT:0407')
selected_collection

<class 'eumdac.collection.Collection'>(EO:EUM:DAT:0407)

In [6]:
# To make the collection output more clear we'll use the 
# HTML module of IPython. It is not mandatory for EUMDAC.
from IPython.core.display import HTML

# Display the details for the selected collection
display(HTML('<b>'+selected_collection.title+'</b>'))
display(HTML('<b>ID:</b> '+str(selected_collection)))
display(HTML('<b>Abstract:</b> '+selected_collection.abstract))

#### Set the area of interest and the dates

In [7]:
# Filter products of selected collection by time and area:

# Add vertices for polygon, wrapping back to the start point.
geometry = [[-0.2, 53.74],
            [0.74, 53.4],
            [0.74, 53.65],
            [-0.2, 53.65],
            [-0.2, 53.74]]

# Set sensing start and end time
# year, month, day, hour, minute
# e.g., 2021-11-10 08:00:00

start = datetime.datetime(2021, 11, 10, 1, 0)
end = datetime.datetime(2021, 11, 11, 23, 59)

print("Start date:" + str(start))
print("Finish date:" + str(end))

# Retrieve datasets that match our filter
products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start, 
    dtend=end)
  
print(f'Found Datasets: {len(products)} datasets for the given time range')

for product in products:
    print(str(product))


Start date:2021-11-10 01:00:00
Finish date:2021-11-11 23:59:00
Found Datasets: 3 datasets for the given time range
S3B_OL_2_WFR____20211111T103808_20211111T104108_20211112T224309_0179_059_108_1980_MAR_O_NT_003.SEN3
S3B_OL_2_WFR____20211110T110419_20211110T110719_20211111T235159_0179_059_094_1980_MAR_O_NT_003.SEN3
S3A_OL_2_WFR____20211110T100242_20211110T100542_20211111T230528_0180_078_236_1980_MAR_O_NT_003.SEN3


#### Download the data


In [39]:
import os

out_folder = r""

for product in products:
    print("Start: " + str(product))
    with product.open() as source:
        out_file = os.path.join(out_folder,str(product) + ".zip")
        print(out_file)
        dest_file = open(out_file,'wb')
    
        shutil.copyfileobj(source,dest_file) 
        print(f'Download of product {product} finished.')
print('All downloads are finished.')

Start: S3B_OL_2_WFR____20211111T103808_20211111T104108_20211112T224309_0179_059_108_1980_MAR_O_NT_003.SEN3
C:\Users\RH19\Desktop\download_test\S3B_OL_2_WFR____20211111T103808_20211111T104108_20211112T224309_0179_059_108_1980_MAR_O_NT_003.SEN3.zip
Start: S3B_OL_2_WFR____20211110T110419_20211110T110719_20211111T235159_0179_059_094_1980_MAR_O_NT_003.SEN3
C:\Users\RH19\Desktop\download_test\S3B_OL_2_WFR____20211110T110419_20211110T110719_20211111T235159_0179_059_094_1980_MAR_O_NT_003.SEN3.zip
Start: S3A_OL_2_WFR____20211110T100242_20211110T100542_20211111T230528_0180_078_236_1980_MAR_O_NT_003.SEN3
C:\Users\RH19\Desktop\download_test\S3A_OL_2_WFR____20211110T100242_20211110T100542_20211111T230528_0180_078_236_1980_MAR_O_NT_003.SEN3.zip
